In [1]:
import pandas as pd
from __future__ import print_function
import pandas as pd
import numpy as np

# matplotlib for displaying the output
import matplotlib.pyplot as plt
import matplotlib.style as ms
ms.use('seaborn-muted')
%matplotlib inline

# and IPython.display for audio output
from IPython.display import Audio
# Librosa for audio
import librosa
# And the display module for visualization
import librosa.display

In [ ]:
filename = '/Users/niki/Documents/255_final_project/ESC-50-master/esc50-human.xlsx'
xl_file = pd.ExcelFile(filename, header = True)
audio_data = xl_file.parse('Raw data')
#audio_data.columns = ['label','audio_file']
audio_data[0:155]

In [ ]:
#removed = original.replace("M", "")
audio_list = []
for name in audio_data.X_AUDIO:
    #print(name)
    new_name = name.replace(".ogg","")
    #print(new_name)
    audio_list.append(new_name)
    
audio_data["X_audio1"]=pd.DataFrame(audio_list)
audio_data.head()

In [ ]:
import os

path = '/Users/niki/Documents/255_final_project/ESC-50-master/audio/'
audio_files = []

for filename in os.listdir(path):
    audio_files.append(filename)
#audio_files


In [ ]:
keyboard = []
for j in audio_data['X_audio1']:
    for name in audio_files:
        if j in name:
            keyboard.append(name)

#print(keyboard)
audio_data["audio_files"]=pd.DataFrame(keyboard)


# audio_data.drop("X_AUDIO", axis=1, inplace=True)

# audio_data.drop("X_audio1", axis=1, inplace=True)


In [ ]:
audio_data.shape

In [ ]:
TRAIN = pd.DataFrame()
TEST = pd.DataFrame()
count=0
c=0

In [ ]:
for i,name in enumerate(audio_data['CLASS_NAME']):
#     c = c+1
    for j in range(0,90):
        if j < 60:
            if count >3938:
                c =1
                break
            data = pd.DataFrame(audio_data.iloc[[count]])
            TRAIN = pd.concat([TRAIN,data], axis=0)
            count = count + 1
            #print ("in if:", j)
        else: 
            if count >3938:
                c =1
                break
            t_data = pd.DataFrame(audio_data.iloc[[count]])
            TEST = pd.concat([TEST,t_data], axis=0)
            count = count + 1
            #print ("in else:", j)
    if c == 1:
        break

In [ ]:
TRAIN.shape

In [ ]:
TRAIN.head()

In [ ]:
TRAIN['audio_files'][60]

In [ ]:
TEST.shape

In [ ]:
TRAIN.to_csv('Audio_Train.csv', index = False)

In [ ]:
abc = pd.read_csv('Audio_Train.csv')

In [ ]:
abc['audio_files'][60:70]

In [ ]:
TEST.to_csv('Audio_Test.csv', index = False)


In [ ]:
def extract_feature(X):
    stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=22050, n_mfcc=40).T,axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=22050).T,axis=0)
    mel = np.mean(librosa.feature.melspectrogram(X, sr=22050).T,axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=22050).T,axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=22050).T,axis=0)
    return mfccs,chroma,mel,contrast,tonnetz

In [ ]:
features= np.empty((0,193))
audio=pd.DataFrame()

In [ ]:
audio = audio.T

In [ ]:
for i in range(1,len(abc)):
    print (i)
    x1 =path + str(abc['audio_files'][i])
    y1, sr1 = librosa.load(x1)
    mfccs, chroma, mel, contrast,tonnetz = extract_feature(y1)
    ext_features = np.hstack([mfccs,chroma,mel,contrast,tonnetz])
    features = np.vstack([features,ext_features])
    X_1=pd.DataFrame(features[i-1])
    audio = pd.concat([audio,X_1], axis=1)

In [ ]:
audio.shape

In [ ]:
audio.head()

In [ ]:
from sklearn import preprocessing

x = audio.values#returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
Xtrain= pd.DataFrame(x_scaled)                

In [ ]:
Xtrain.to_csv('audio_features.csv',index=False)

In [ ]:
xyz = pd.read_csv('Audio_Test.csv')

In [ ]:
xyz.head()

In [ ]:
test_features= np.empty((0,193))
test_audio=pd.DataFrame()

for i in range(1,len(xyz)):
    x2 =path + str(xyz['audio_files'][i])
    y2, sr2 = librosa.load(x2)
    mfccs, chroma, mel, contrast,tonnetz = extract_feature(y2)
    test_ext_features = np.hstack([mfccs,chroma,mel,contrast,tonnetz])
    test_features = np.vstack([test_features,test_ext_features])
    test_X_1=pd.DataFrame(test_features[i-1])
    test_audio = pd.concat([test_audio,test_X_1], axis=1)

In [ ]:
test_audio.shape

In [ ]:
test_audio = test_audio.T

In [ ]:
test_audio.head()

In [ ]:

xT = test_audio.values#returns a numpy array
#min_max_scaler = preprocessing.MinMaxScaler()
Tx_scaled = min_max_scaler.fit_transform(xT)
Xtest= pd.DataFrame(Tx_scaled)   

In [ ]:
Xtest.to_csv('Test_audio_features.csv',index=False)

### tesnsor flow

In [2]:
Xtrain = pd.read_csv('audio_features.csv') 

In [36]:
Xtest = pd.read_csv('Test_audio_features.csv')

In [37]:
x = pd.read_csv('Audio_Train.csv')
x.head()

,CLASS_NAME,X_AUDIO,Y_CLASS,X_audio1,audio_files
0,Airplane,1-11687-A.ogg,1,1-11687-A,1-11687-A-47.wav
1,Airplane,1-24796-A.ogg,1,1-24796-A,1-24796-A-47.wav
2,Airplane,1-36929-A.ogg,1,1-36929-A,1-36929-A-47.wav
3,Airplane,1-43807-A.ogg,1,1-43807-A,1-43807-A-47.wav
4,Airplane,1-43807-B.ogg,1,1-43807-B,1-43807-B-47.wav


In [38]:
Ytrain = x['Y_CLASS']

In [39]:
y = pd.read_csv('Audio_Test.csv')

Ytest = y['Y_CLASS']

In [40]:
print(Xtrain.shape)
Ytrain=Ytrain[1:]
len(Ytrain)

(2639, 193)


2639

In [41]:
# from sklearn import preprocessing

# x = train_data.values#returns a numpy array
# min_max_scaler = preprocessing.MinMaxScaler()
# x_scaled = min_max_scaler.fit_transform(x)
# train_df = pd.DataFrame(x_scaled)   

In [42]:
# Xtrain = train_df
# Ytrain = labels
 

In [43]:
# xT = test_audio.values#returns a numpy array
# #min_max_scaler = preprocessing.MinMaxScaler()
# x_scaledT = min_max_scaler.fit_transform(xT)
# Xtest = pd.DataFrame(x_scaledT)   


In [44]:
# print(Xtrain.shape[0])

In [74]:
import tensorflow as tf

training_epochs = 2639
dim = 193
classnum = 2639
hidden1 = 300 
hidden2 = 300
# standard deviation
sd = 1 / np.sqrt(dim) 
learning_rate = 0.01

In [75]:
X = tf.placeholder(tf.float32,[None,dim]) 
Y = tf.placeholder(tf.float32,[classnum,])

In [76]:
# Weight and bias
# TANH function in the first hidden layer
W1 = tf.Variable(tf.random_normal([dim,hidden1], mean = 0, stddev=sd))
b1 = tf.Variable(tf.random_normal([hidden1], mean = 0, stddev=sd))
hf1 = tf.nn.tanh(tf.matmul(X,W1) + b1)

# Sigmoid function in the second hidden layer
W2 = tf.Variable(tf.random_normal([hidden1,hidden2], mean = 0, stddev=sd))
b2 = tf.Variable(tf.random_normal([hidden2], mean = 0, stddev=sd))
hf2 = tf.nn.sigmoid(tf.matmul(hf1,W2) + b2)

# Softmax function in output layer
W = tf.Variable(tf.random_normal([hidden2,classnum], mean = 0, stddev=sd))
b = tf.Variable(tf.random_normal([classnum], mean = 0, stddev=sd))
finalY = tf.nn.softmax(tf.matmul(hf2,W) + b)

In [77]:
# # Summary Operations
# wHisto=tf.histogram_summary("Weights",W)
# bHisto=tf.histogram_summary("Biases",b)

In [78]:
init = tf.global_variables_initializer()

In [79]:
#Minimise error - Cross Entropy Function
cost_function = -tf.reduce_sum(Y * tf.log(finalY))
tf.summary.scalar("Cost_Function", cost_function)

optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_function)
# Defining and initializing variables for accuracy calculation 
correct_prediction = tf.equal(tf.argmax(finalY,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


In [80]:
cost_history = np.empty(shape=[1],dtype=float)
y_true, y_pred = None, None
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(training_epochs):            
        _,cost = sess.run([optimizer,cost_function],feed_dict={X:Xtrain,Y:Ytrain})
        cost_history = np.append(cost_history,cost)
    
    y_pred = sess.run(tf.argmax(finalY,1),feed_dict={X: Xtest})
    y_true = sess.run(tf.argmax(Ytest,1))
#     print('Test accuracy: ',round(session.run(accuracy, feed_dict={X: Xtest, Y: Ytest},3)))

fig = plt.figure(figsize=(10,8))
plt.plot(cost_history)
plt.axis([0,training_epochs,0,np.max(cost_history)])
plt.show()

p,r,f,s = precision_recall_fscore_support(y_true, y_pred, average='micro')
print("F-Score:", round(f,3))


ValueError: Tried to convert 'input' to a tensor and failed. Error: Argument must be a dense tensor: 0        1
1        1
2        1
3        1
4        1
5        1
6        1
7        1
8        1
9        1
10       1
11       1
12       1
13       1
14       1
15       1
16       1
17       1
18       2
19       2
20       2
21       2
22       2
23       2
24       2
25       2
26       2
27       2
28       2
29       2
        ..
1269    49
1270    49
1271    49
1272    49
1273    49
1274    49
1275    49
1276    50
1277    50
1278    50
1279    50
1280    50
1281    50
1282    50
1283    50
1284    50
1285    50
1286    50
1287    50
1288    50
1289    50
1290    50
1291    50
1292    50
1293    50
1294    50
1295    50
1296    50
1297    50
1298    50
Name: Y_CLASS, Length: 1299, dtype: int64 - got shape [1299], but wanted [].

In [65]:
#Merge all summaries into single operator
mergedSummary=tf.summary.merge_all()


In [ ]:
# Defining and initializing variables for accuracy calculation 
correct_prediction = tf.equal(tf.argmax(finalY,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
Ytest = xyz['Y_CLASS']

In [82]:
listmy=[2,3,4]
min(listmy)

2